In [1]:
from pprint import pprint
import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick
# import seaborn as sns
import datetime as dt

In [2]:
import jetstream
from jetstream.experimenter import ExperimentCollection
from jetstream.external_config import ExternalConfigCollection, ExternalConfig, ExternalOutcome, OUTCOMES_DIR, DEFAULTS_DIR, ExternalDefaultConfig
from jetstream.cli import AnalysisExecutor, run, SerialExecutorStrategy, All
from jetstream import AnalysisPeriod
from jetstream.bigquery_client import BigQueryClient
from jetstream.util import TemporaryDirectory
from jetstream.config import AnalysisSpec, OutcomeSpec
from jetstream.experimenter import Experiment as JExperiment
from git import Repo
import toml
from pytz import UTC

In [3]:
from mozanalysis.bq import BigQueryContext
from mozanalysis.experiment import Experiment
from mozanalysis.metrics import Metric
from mozanalysis.metrics.desktop import main, events
from mozanalysis.exposure import ExposureSignal
from mozanalysis.frequentist_stats.bootstrap import compare_branches

# Monkey patch the config loader to use our jetstream-config branch

In [4]:
@classmethod
def from_github_repo(cls) -> "ExternalConfigCollection":
    """Pull in external config files."""
    # download files to tmp directory
    with TemporaryDirectory() as tmp_dir:
        repo = Repo.clone_from(cls.JETSTREAM_CONFIG_URL, tmp_dir)
        #changed line here
        repo.git.checkout('add_search_count_for_fenix')
        external_configs = []

        for config_file in tmp_dir.glob("*.toml"):
            last_modified = next(repo.iter_commits("main", paths=config_file)).committed_date

            external_configs.append(
                ExternalConfig(
                    config_file.stem,
                    AnalysisSpec.from_dict(toml.load(config_file)),
                    UTC.localize(dt.datetime.utcfromtimestamp(last_modified)),
                )
            )

        outcomes = []

        for outcome_file in tmp_dir.glob(f"**/{OUTCOMES_DIR}/*/*.toml"):
            commit_hash = next(repo.iter_commits("main", paths=outcome_file)).hexsha

            outcomes.append(
                ExternalOutcome(
                    slug=outcome_file.stem,
                    spec=OutcomeSpec.from_dict(toml.load(outcome_file)),
                    platform=outcome_file.parent.name,
                    commit_hash=commit_hash,
                )
            )

        default_configs = []
        for default_config_file in tmp_dir.glob(f"**/{DEFAULTS_DIR}/*.toml"):
            last_modified = next(
                repo.iter_commits("main", paths=default_config_file)
            ).committed_date

            default_configs.append(
                ExternalDefaultConfig(
                    default_config_file.stem,
                    AnalysisSpec.from_dict(toml.load(default_config_file)),
                    UTC.localize(dt.datetime.utcfromtimestamp(last_modified)),
                )
            )

    return cls(external_configs, outcomes, default_configs)

In [5]:
ExternalConfigCollection.from_github_repo = from_github_repo

# Configure mozanalysis/jetstream classes

In [6]:
# Each experiment has an enrollment period and an observation period.
# We analyze data from each Firefox client *relative* to their enrollment date, as opposed to an absolute calendar date.
NUM_DATES_ENROLLMENT = 7
ANALYSIS_WINDOW_LENGTH = 7
NUM_ANALYSIS_WINDOWS = 3

In [7]:
PROJECT_ID = 'moz-fx-data-bq-data-science'
DATASET_ID = 'dberry'

In [8]:
# Experiment set up
bq_client = BigQueryClient(PROJECT_ID, DATASET_ID)

bq_context = BigQueryContext(
  dataset_id=DATASET_ID,  # BQ tables will be created in this namespace
  project_id=PROJECT_ID
)

experiment = Experiment(
  experiment_slug='android-nightly-sponsored-shortcuts-validation',  # unique ID for Nimbus experiment
  start_date='2022-03-16',
  num_dates_enrollment = NUM_DATES_ENROLLMENT
)
  

In [9]:
ae = AnalysisExecutor(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    bucket='dberry-simulated-aa-tests-temporary',
    date=All,
    experiment_slugs=[experiment.experiment_slug]
)

# Generate the run configs
Here we're manually executing the steps that jetstream would do to build the job, just to confirm that they're being picked up

In [10]:
run_configs = ae._experiment_configs_to_analyse(ExperimentCollection.from_experimenter, ExternalConfigCollection.from_github_repo)

In [11]:
metrics_week = set([summary.metric.to_mozanalysis_metric() for summary in run_configs[0].metrics[AnalysisPeriod.WEEK]])

In [12]:
metrics_overall = set([summary.metric.to_mozanalysis_metric() for summary in run_configs[0].metrics[AnalysisPeriod.OVERALL]])

In [20]:
# you should see 'search_count' and 'serp_ad_clicks' here now

In [13]:
[m.name for m in metrics_week]

['total_uri_count',
 'retained',
 'serp_ad_clicks',
 'days_of_use',
 'tagged_sap_searches',
 'search_count',
 'active_hours']

In [14]:
[m.name for m in metrics_overall]

['tagged_follow_on_searches',
 'searches_with_ads',
 'total_uri_count',
 'serp_ad_clicks',
 'days_of_use',
 'search_count',
 'active_hours',
 'organic_searches']

# Rerun analysis

In [15]:
ae.execute(
    strategy=SerialExecutorStrategy(PROJECT_ID, DATASET_ID, ae.bucket)
)

2022-04-11 16:44:50,335 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/dberry/jetstream-config_2022_04_11_ANSSV_debugging/dask-worker-space/worker-opi7kb8y', purging
2022-04-11 16:44:50,336 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/dberry/jetstream-config_2022_04_11_ANSSV_debugging/dask-worker-space/worker-mpfvd1tq', purging
2022-04-11 16:44:50,336 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/dberry/jetstream-config_2022_04_11_ANSSV_debugging/dask-worker-space/worker-fqh7_fi0', purging
2022-04-11 16:44:50,336 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/dberry/jetstream-config_2022_04_11_ANSSV_debugging/dask-worker-space/worker-vu637d1o', purging
2022-04-11 16:44:50,336 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/dberry/jetstream-config_2022_04_11_ANSSV_debugging/dask-worker-space/worker-rulf48kv', purging
2022-04-11 16:44:50,

True

# Get analysis results

In [16]:
experiment_reanalysis_metrics_1 = bq_client.table_to_dataframe('statistics_android_nightly_sponsored_shortcuts_validation_week_1')
experiment_reanalysis_metrics_1.loc[:, 'week'] = 1
experiment_reanalysis_metrics_2 = bq_client.table_to_dataframe('statistics_android_nightly_sponsored_shortcuts_validation_week_2')
experiment_reanalysis_metrics_2.loc[:, 'week'] = 2
experiment_reanalysis_metrics = pd.concat([experiment_reanalysis_metrics_1, experiment_reanalysis_metrics_2])

# Compare against UI

In [17]:
metrics_vs_prod = (
    experiment_reanalysis_metrics
    .loc[(
        experiment_reanalysis_metrics.metric.isin(['days_of_use', 'search_count']) & 
        (experiment_reanalysis_metrics.statistic == 'mean') &
        (experiment_reanalysis_metrics.comparison == 'relative_uplift')
    )]
    .sort_values(['metric', 'week'])    
    [['metric', 'week', 'lower', 'upper']]
)
metrics_vs_prod.loc[:, 'lower'] = (metrics_vs_prod.lower*100).round(1)
metrics_vs_prod.loc[:, 'upper'] = (metrics_vs_prod.upper*100).round(1)

In [18]:
# days_of_use should approximately equal to https://experimenter.services.mozilla.com/nimbus/android-nightly-sponsored-shortcuts-validation/results#results_summary
# some slight deviation is expected due to the random nature of the bootstrap confidence intervals

In [19]:
metrics_vs_prod

,metric,week,lower,upper
637,days_of_use,1,-1.6,1.0
637,days_of_use,2,-2.3,1.3
649,search_count,1,-4.7,2.4
648,search_count,2,-5.0,2.8
